## Example with Portfolio
This example is used for demonstration on a Portfolio. Starting with the data acquisation phase. For this example data is downloaded from nsepy library. 

In [1]:
from nsepy import get_history
from datetime import date

In [2]:
start_date = date(2010,1,1)
end_date = date.today()

In [3]:
PORTFOLIO = ["TATACONSUM", "RELIANCE", "WIPRO", "TECHM", "BHARTIARTL"]

In [4]:
def convert_to_kaggle_format(data):
    import pandas
    
    new_dataset = data[['Open','High', 'Low', 'Close', 'Volume', 'Symbol']]
    new_dataset.columns = ['Open','High', 'Low', 'Close', 'Volume', 'Name']
    
    export_name = data["Symbol"][0]+ '_' + str(start_date) + str(end_date) + '.csv'
    
    return new_dataset, export_name

In [5]:
# Create or Overwrite in a new folder to save data.
import os, shutil

if not(os.path.exists("PORTFOLIO")):
    os.mkdir("PORTFOLIO")
else:
    shutil.rmtree("PORTFOLIO")
    os.mkdir("PORTFOLIO")

In [6]:
def get_data_of_portfolio(p=PORTFOLIO):
    for point in p:
        data = get_history(symbol=point, start=start_date, end=end_date)
        ax = data[['Close']].plot()
        ax.title.set_text(data["Symbol"][0])
        new_dataset, export_name = convert_to_kaggle_format(data)
        export_name = os.path.join("PORTFOLIO", export_name)
        new_dataset.to_csv(export_name)

In [7]:
get_data_of_portfolio(PORTFOLIO)

## Training and prediction phase
The training and prediction is done using LSTM and GRU models. For this, we use pytorch library.

In [8]:
import torch
import numpy as np

from Models import LSTM
from Models import GRU

import time
from sklearn.preprocessing import MinMaxScaler

import HelperFunctions as H

In [9]:
#Hyper Parfameters
input_dim = 1
hidden_dim = 32
num_layers = 2
output_dim = 1
num_epochs = 100

lookback = 20 # choose sequence length


In [10]:
def load_to_torch(x_train, x_test, y_train, y_test):
    x_train = torch.from_numpy(x_train).type(torch.Tensor)
    x_test = torch.from_numpy(x_test).type(torch.Tensor)
    y_train_lstm = torch.from_numpy(y_train).type(torch.Tensor)
    y_test_lstm = torch.from_numpy(y_test).type(torch.Tensor)
    y_train_gru = torch.from_numpy(y_train).type(torch.Tensor)
    y_test_gru = torch.from_numpy(y_test).type(torch.Tensor)
    return x_train, x_test, y_train_lstm, y_test_lstm, y_train_gru, y_test_gru

In [11]:
def train_lstm(x_train, y_train_lstm):
    lstm_model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
    criterion = torch.nn.MSELoss(reduction='mean')
    optimiser = torch.optim.Adam(lstm_model.parameters(), lr=0.01)

    hist = np.zeros(num_epochs)
    start_time = time.time()
    lstm = []

    for t in range(num_epochs):
        y_train_pred = lstm_model(x_train)

        loss = criterion(y_train_pred, y_train_lstm)
        print("Epoch ", t, "MSE: ", loss.item())
        hist[t] = loss.item()

        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
        
    training_time = time.time()-start_time
    print("Training time: {}".format(training_time))

    return lstm_model

In [12]:
def train_gru(x_train, y_train_gru):
    gru_model = GRU(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
    criterion = torch.nn.MSELoss(reduction='mean')
    optimiser = torch.optim.Adam(gru_model.parameters(), lr=0.01)

    hist = np.zeros(num_epochs)
    start_time = time.time()
    gru = []

    for t in range(num_epochs):
        y_train_pred = gru_model(x_train)

        loss = criterion(y_train_pred, y_train_gru)
        print("Epoch ", t, "MSE: ", loss.item())
        hist[t] = loss.item()

        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

    training_time = time.time()-start_time    
    print("Training time: {}".format(training_time))

    return gru_model

In [13]:
def predict_next_day(model, price, lookback, scaler):
    print("Using the Model . . . . . : ", model)

    #scaler = MinMaxScaler(feature_range=(-1, 1))

    data_all = price.to_numpy() # convert to numpy array
    last_data = []

    last_data.append(data_all[-(lookback-1):])

    last_data = np.array(last_data)
    x_last = torch.from_numpy(last_data).type(torch.Tensor)

    pred = model(x_last)
    pred = scaler.inverse_transform(pred.detach().numpy())

    print("The predicted closing price of the stock on next day in INR is: ", pred[0][0])
    return(pred[0][0])

Go through the stocks and predict. 

In [14]:
portfolio_list = os.listdir("PORTFOLIO")

LSTM_preds = []
GRU_preds = []

for each in portfolio_list:
    filename = os.path.join("PORTFOLIO", each)

    price, scaler = H.load_data(filename)
    x_train, y_train, x_test, y_test = H.split_data(price, lookback)
    x_train, x_test, y_train_lstm, y_test_lstm, y_train_gru, y_test_gru = \
        load_to_torch(x_train, x_test, y_train, y_test)
    lstm_model = train_lstm(x_train, y_train_lstm)
    gru_model = train_gru(x_train, y_train_gru)
    
    LSTM_preds.append(predict_next_day(lstm_model, price, lookback, scaler))
    GRU_preds.append(predict_next_day(gru_model, price, lookback, scaler))


/data3/Akram/Projects/SafeMLTimeSeries/TimeSeriesSafeML/LSTM_Stock/HelperFunctions.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['Close'] = scaler.fit_transform(price['Close'].values.reshape(-1,1))


Epoch  0 MSE:  0.15339085459709167
Epoch  1 MSE:  0.1406661421060562
Epoch  2 MSE:  0.12085021287202835
Epoch  3 MSE:  0.09313173592090607
Epoch  4 MSE:  0.03811807930469513
Epoch  5 MSE:  0.03366374969482422
Epoch  6 MSE:  0.08548569679260254
Epoch  7 MSE:  0.010392501950263977
Epoch  8 MSE:  0.017489969730377197
Epoch  9 MSE:  0.03493088111281395
Epoch  10 MSE:  0.04068869352340698
Epoch  11 MSE:  0.04015260189771652
Epoch  12 MSE:  0.036685746163129807
Epoch  13 MSE:  0.03071688301861286
Epoch  14 MSE:  0.02212604135274887
Epoch  15 MSE:  0.012598767876625061
Epoch  16 MSE:  0.008113264106214046
Epoch  17 MSE:  0.014180655591189861
Epoch  18 MSE:  0.019563697278499603
Epoch  19 MSE:  0.016877735033631325
Epoch  20 MSE:  0.013965495862066746
Epoch  21 MSE:  0.011807232163846493
Epoch  22 MSE:  0.009188102558255196
Epoch  23 MSE:  0.0073906551115214825
Epoch  24 MSE:  0.00767082953825593
Epoch  25 MSE:  0.009522509761154652
Epoch  26 MSE:  0.011043617501854897
Epoch  27 MSE:  0.010675

/data3/Akram/Projects/SafeMLTimeSeries/TimeSeriesSafeML/LSTM_Stock/HelperFunctions.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['Close'] = scaler.fit_transform(price['Close'].values.reshape(-1,1))


Epoch  1 MSE:  0.28273120522499084
Epoch  2 MSE:  0.19646359980106354
Epoch  3 MSE:  0.20531238615512848
Epoch  4 MSE:  0.16695858538150787
Epoch  5 MSE:  0.1402388960123062
Epoch  6 MSE:  0.13092580437660217
Epoch  7 MSE:  0.11330781877040863
Epoch  8 MSE:  0.08107800036668777
Epoch  9 MSE:  0.06200624629855156
Epoch  10 MSE:  0.05913032591342926
Epoch  11 MSE:  0.027371525764465332
Epoch  12 MSE:  0.033618468791246414
Epoch  13 MSE:  0.0292644239962101
Epoch  14 MSE:  0.02195356972515583
Epoch  15 MSE:  0.03806532919406891
Epoch  16 MSE:  0.03095288574695587
Epoch  17 MSE:  0.01928502693772316
Epoch  18 MSE:  0.01829536072909832
Epoch  19 MSE:  0.0180748850107193
Epoch  20 MSE:  0.014929130673408508
Epoch  21 MSE:  0.012633934617042542
Epoch  22 MSE:  0.014241045340895653
Epoch  23 MSE:  0.01684652641415596
Epoch  24 MSE:  0.016064302995800972
Epoch  25 MSE:  0.012886934913694859
Epoch  26 MSE:  0.010538584552705288
Epoch  27 MSE:  0.009715663269162178
Epoch  28 MSE:  0.0094514125958

/data3/Akram/Projects/SafeMLTimeSeries/TimeSeriesSafeML/LSTM_Stock/HelperFunctions.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['Close'] = scaler.fit_transform(price['Close'].values.reshape(-1,1))


Epoch  1 MSE:  0.5085919499397278
Epoch  2 MSE:  0.23808209598064423
Epoch  3 MSE:  0.019499948248267174
Epoch  4 MSE:  0.4588162302970886
Epoch  5 MSE:  0.1272438019514084
Epoch  6 MSE:  0.020096268504858017
Epoch  7 MSE:  0.05637551099061966
Epoch  8 MSE:  0.09887867420911789
Epoch  9 MSE:  0.11888096481561661
Epoch  10 MSE:  0.11924225091934204
Epoch  11 MSE:  0.10691913217306137
Epoch  12 MSE:  0.08802656084299088
Epoch  13 MSE:  0.06741947680711746
Epoch  14 MSE:  0.04882403090596199
Epoch  15 MSE:  0.03485635668039322
Epoch  16 MSE:  0.026907868683338165
Epoch  17 MSE:  0.025019118562340736
Epoch  18 MSE:  0.02790878154337406
Epoch  19 MSE:  0.03331369534134865
Epoch  20 MSE:  0.038685526698827744
Epoch  21 MSE:  0.042031168937683105
Epoch  22 MSE:  0.042479392141103745
Epoch  23 MSE:  0.0403163768351078
Epoch  24 MSE:  0.03658029064536095
Epoch  25 MSE:  0.032508235424757004
Epoch  26 MSE:  0.0290994793176651
Epoch  27 MSE:  0.02690819464623928
Epoch  28 MSE:  0.0260415542870759

/data3/Akram/Projects/SafeMLTimeSeries/TimeSeriesSafeML/LSTM_Stock/HelperFunctions.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['Close'] = scaler.fit_transform(price['Close'].values.reshape(-1,1))


Epoch  1 MSE:  0.7351824045181274
Epoch  2 MSE:  0.4444396197795868
Epoch  3 MSE:  0.11610980331897736
Epoch  4 MSE:  0.32289960980415344
Epoch  5 MSE:  0.2173181027173996
Epoch  6 MSE:  0.08452164381742477
Epoch  7 MSE:  0.0745820701122284
Epoch  8 MSE:  0.11037614941596985
Epoch  9 MSE:  0.1318502575159073
Epoch  10 MSE:  0.12930040061473846
Epoch  11 MSE:  0.10999880731105804
Epoch  12 MSE:  0.0840316116809845
Epoch  13 MSE:  0.059929970651865005
Epoch  14 MSE:  0.0423743799328804
Epoch  15 MSE:  0.031691983342170715
Epoch  16 MSE:  0.02674373798072338
Epoch  17 MSE:  0.027727866545319557
Epoch  18 MSE:  0.031583257019519806
Epoch  19 MSE:  0.033120039850473404
Epoch  20 MSE:  0.030782029032707214
Epoch  21 MSE:  0.026225844398140907
Epoch  22 MSE:  0.021824857220053673
Epoch  23 MSE:  0.01916412264108658
Epoch  24 MSE:  0.01861378364264965
Epoch  25 MSE:  0.019600672647356987
Epoch  26 MSE:  0.021134013310074806
Epoch  27 MSE:  0.02226579189300537
Epoch  28 MSE:  0.0223775021731853

/data3/Akram/Projects/SafeMLTimeSeries/TimeSeriesSafeML/LSTM_Stock/HelperFunctions.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['Close'] = scaler.fit_transform(price['Close'].values.reshape(-1,1))


Epoch  1 MSE:  0.18221350014209747
Epoch  2 MSE:  0.060263458639383316
Epoch  3 MSE:  0.06302431970834732
Epoch  4 MSE:  0.04501436650753021
Epoch  5 MSE:  0.027303554117679596
Epoch  6 MSE:  0.03378516808152199
Epoch  7 MSE:  0.04015734791755676
Epoch  8 MSE:  0.03945304453372955
Epoch  9 MSE:  0.034418728202581406
Epoch  10 MSE:  0.02929508313536644
Epoch  11 MSE:  0.0271406602114439
Epoch  12 MSE:  0.028484808281064034
Epoch  13 MSE:  0.03103623539209366
Epoch  14 MSE:  0.03178597614169121
Epoch  15 MSE:  0.029977791011333466
Epoch  16 MSE:  0.02725948765873909
Epoch  17 MSE:  0.025484269484877586
Epoch  18 MSE:  0.02521093189716339
Epoch  19 MSE:  0.025808585807681084
Epoch  20 MSE:  0.026267800480127335
Epoch  21 MSE:  0.025879476219415665
Epoch  22 MSE:  0.024534326046705246
Epoch  23 MSE:  0.022696934640407562
Epoch  24 MSE:  0.021124189719557762
Epoch  25 MSE:  0.020357998088002205
Epoch  26 MSE:  0.02016175352036953
Epoch  27 MSE:  0.019489720463752747
Epoch  28 MSE:  0.017544

In [15]:
print("LSTM predictions are as follows : ")
for s, p in zip(portfolio_list, LSTM_preds):
    print("For stock ", s, " prize in INR is ", p)

LSTM predictions are as follows : 
For stock  WIPRO_2010-01-012022-01-09.csv  prize in INR is  708.7388
For stock  TECHM_2010-01-012022-01-09.csv  prize in INR is  1723.0642
For stock  RELIANCE_2010-01-012022-01-09.csv  prize in INR is  1990.1561
For stock  TATATEA_2010-01-012022-01-09.csv  prize in INR is  844.0664
For stock  BHARTIARTL_2010-01-012022-01-09.csv  prize in INR is  604.27637


In [16]:
print("GRU predictions are as follows : ")
for s, p in zip(portfolio_list, GRU_preds):
    print("For stock ", s, " prize in INR is ", p)

GRU predictions are as follows : 
For stock  WIPRO_2010-01-012022-01-09.csv  prize in INR is  703.8467
For stock  TECHM_2010-01-012022-01-09.csv  prize in INR is  1723.4923
For stock  RELIANCE_2010-01-012022-01-09.csv  prize in INR is  2283.8984
For stock  TATATEA_2010-01-012022-01-09.csv  prize in INR is  755.6422
For stock  BHARTIARTL_2010-01-012022-01-09.csv  prize in INR is  665.85834
